In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [4]:
dataset_path = './dataset'
dataset = datasets.ImageFolder(root = dataset_path,transform=image_transforms)

In [5]:
n_classes = len(dataset.classes)

In [6]:
train_size = int(0.75 * len(dataset))
train_size

1725

In [7]:
val_size = len(dataset) - train_size

In [8]:
from torch.utils.data import random_split
train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

In [9]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(val_dataset,batch_size=32,shuffle=False)

In [11]:
class ResnetClassifier(nn.Module):
    def __init__(self,num_classes,dropout_rate):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.layer4.parameters():
            param.requires_grad = True
        
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features,num_classes)
        )

    def forward(self,x):
        x = self.model(x)
        return x

In [12]:
import optuna


In [13]:

def objective(trial):
    
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    
    model = ResnetClassifier(num_classes=6, dropout_rate=dropout_rate).to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        
        trial.report(accuracy, epoch)
        
        
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-07-15 12:23:13,411] A new study created in memory with name: no-name-53d7777c-95a9-4702-8d57-6e1ae46c64ef
[I 2025-07-15 12:30:37,931] Trial 0 finished with value: 56.289855072463766 and parameters: {'lr': 1.3078136424010068e-05, 'dropout_rate': 0.4867651993753625}. Best is trial 0 with value: 56.289855072463766.


Execution time: 443.70324301719666 seconds


[I 2025-07-15 12:39:34,445] Trial 1 finished with value: 95.1304347826087 and parameters: {'lr': 0.0004107126928527431, 'dropout_rate': 0.3012286226300925}. Best is trial 1 with value: 95.1304347826087.


Execution time: 535.0554733276367 seconds


[I 2025-07-15 12:43:56,775] Trial 2 finished with value: 94.6086956521739 and parameters: {'lr': 0.0003386884780237862, 'dropout_rate': 0.5583789751818813}. Best is trial 1 with value: 95.1304347826087.


...

Execution time: 262.0659182071686 seconds


[I 2025-07-15 13:11:39,204] Trial 11 pruned. 
[I 2025-07-15 13:14:29,199] Trial 12 pruned. 
[I 2025-07-15 13:18:51,707] Trial 13 finished with value: 95.07246376811594 and parameters: {'lr': 0.00014667313520859464, 'dropout_rate': 0.3278178196999911}. Best is trial 1 with value: 95.1304347826087.


Execution time: 262.1841666698456 seconds


[I 2025-07-15 13:23:10,981] Trial 14 pruned. 
[I 2025-07-15 13:27:33,487] Trial 15 pruned. 
[I 2025-07-15 13:30:28,151] Trial 16 pruned. 
[I 2025-07-15 13:31:55,501] Trial 17 pruned. 
[I 2025-07-15 13:34:46,499] Trial 18 pruned. 
[I 2025-07-15 13:38:59,866] Trial 19 finished with value: 96.40579710144928 and parameters: {'lr': 0.00022857126729455005, 'dropout_rate': 0.40297443917883574}. Best is trial 19 with value: 96.40579710144928.


Execution time: 253.0286774635315 seconds


In [15]:
study.best_params

{'lr': 0.00022857126729455005, 'dropout_rate': 0.40297443917883574}